# Лекция 5: Обучение с подкреплением

__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2023 г. 

При подготовке лекции использованы материалы:
* ...

v 0.3 01.11.23

## Разделы: <a class="anchor" id="разделы"></a>
* [раздел 1](#загрузка)
* [раздел 2](#нормализация)
-

* [к оглавлению](#разделы)


In [1]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v2.css")
HTML(html.read().decode('utf-8'))

## Введение в обучение с подкреплением

__Различные подходы к машинному обучению__

<br/>
<center>         
    <img src="./img/l_types1.png" alt="" style="width: 700px;"/>
    <b>Типы машинного обучения (Часть 1)</b> <br/>
</center>

__Обучение с учителем__ (Supervised Learning) - модель обучается __на размеченных данных__, где каждый входной пример имеет соответствующий целевой выход.
* __Цель__ состоит в том, чтобы модель научилась __предсказывать целевой выход для новых__, ранее не виденных __данных__.
* __Задачи__: классификация и регрессия.

__Обучение без учителя__ (Unsupervised Learning) - модель обучается на неразмеченных данных, где отсутствует целевой выход.
* __Цель__ - найти внутренние структуры, закономерности или группировки в данных.
* __Задачи__: кластеризация, выявление аномалий, понижение размерности, поиск ассоциативных правил.

__Обучение с частичным привлечением учителя__ (Semi-Supervised Learning) - комбинация обучения с учителем и обучения без учителя, где модель обучается на __небольшом наборе размеченных данных__ и __большем наборе неразмеченных данных__.
* __Цель и задачи__ аналогичны обучению с учителем, __идея__ заключается в использовании неразмеченных данных для улучшения обучения модели и расширения ее способности к обобщению.

__Самообучение__ (Self-Supervised Learning) - подходк к обучению, в котором модель обучается на неразмеченных данных, используя разнообразные задачи, создаваемые из самих данных, без необходимости внешних разметок. Вместо того, чтобы полагаться на учителя для предоставления меток (как в обучении с учителем), Self-Supervised Learning использует информацию, извлекаемую из данных, для самостоятельного обучения модели.
* __Цель и задачи__ аналогичны обучению с учителем, входые данные - аналогичны обучению без учителя. (см.: https://dasha.ai/en-us/blog/self-supervised-machine-learning )


<br/>
<center>         
    <img src="./img/act_learning.png" alt="" style="width: 350px;"/>
    <b>Принцип работы активного обучения</b> <br/>
</center>

__Активное обучение (Active Learning)__ - метод, применяемый в подходе __обучения с учителем__, обычно в случаях __когда получение меток дорого__, поэтому мы получаем __новые метки динамически__, определяя алгоритмическую __стратегию для максимизации полезности новых наблюдений__. 
* Можно рассматривать активное обучение как __особый случай обучения с частичным привлечением учителя__.
* __Цель и задачи__ аналогичны обучению с учителем, входые данные - без разметки, с возможностью (обычно дорогстоящей) динамической разметки.

<br/>
<center>         
    <img src="./img/rl1.png" alt="" style="width: 500px;"/>
    <b>Принцип работы обучения с подкреплением</b> <br/>
</center>

__Обучение с подкреплением (Reinforcement Learning, RL)__ - модель обучается как __агент, взаимодействующий с окружающей средой__ и максимизируют вознаграждение получаемое от системы.
* В RL агент самостоятельно исследует окружающую среду, принимая решения на основе своего опыта и обратной связи от среды. 
* В RL нет меток:
    * нельзя использовать обучение с учителем
    * есть "вознаграждение", которое говорит насколько хорош текущий результат.Следовательно,
* В RL модель __учится стратегии__ максимизации вознаграждения.
    * стратегия должна должна учитывать баланс между исследованием (получением новый информации о среде) и использованием существующих знаний о ней для максимизации вознаграждения.


__Приложения обученяи с подкреплением__

<br/>
<center>         
    <img src="./img/rl_ap1.png" alt="" style="width: 650px;"/>
    <b>Приложения обученяи с подкреплением</b> <br/>
</center>


__Постановка задачи обучения с подкреплением__

__Обучение с подкреплением__ (reinforcement learning) — метод машинного обучения, при котором система обучается, взаимодействуя с некоторой средой. В обучении с подкреплением есть:
* __агент (agent)__ кторый при помощи
* __действий (actions)__ взаимодействует с 
* окружающей __средой (environment)__ описываемой
* внутренним __состоянием (state)__
* в ответ на действие среда возвращает __вознаграждение (reward)__ за эти действия
* и меняет свое сотоятение.

<br/>
<center>         
    <img src="./img/rl1.png" alt="" style="width: 500px;"/>
    <b>Принцип работы обучения с подкреплением</b> <br/>
</center>

* $s_t$ - сотояние среды (state) на шаге $t$
* $a_t$ - действия агента (agent) на шаге $t$
* $r_t$ - вознаграждение (reward) на шаге $t$

---
* $s \in S$ - сотояния
* $a \in A$ - действия
* $p(s'|a, s)$ - модель переходов между состояниями (в общем случае - вероятностная)
    * модель переходов неизвестна агенту и в зависимости от подхода к решению задачи агент может пытаться, а может не пытаться узнать модель переходов.
* $\pi(a|s)$ - стратегия (policy, политика) 
* $r(s, a)$ - вознаграждение
    * для многих задач построение функции вознаграждения сложная задача. Пример: шахматы.

__Стратегия в RL__

__Стратегия__ (policy, политика) в RL - это стратегия или набор правил, которые определяют, какое действие должен выбирать агент должен при заданном состоянии среды.
* Формально может быть записано как распределение: $\pi(a|s)$
* Статегия определяет, как агент будет взаимодействовать с окружающей средой, и важна для достижения целей обучения с подкреплением.

Стратегия может быть определена различными способами:
* __Детерминированная стратегия__ -  для каждого состояния среды существует конкретное действие, которое агент должен выполнить. 
    * Например, если агент находится в состоянии A, он всегда выполняет действие X.
* __Случайная стратегия__ - агент выбирает действия с определенными вероятностями для каждого состояния.
* __Параметризованная стратегия__ - например, стратегия может быть параметризованной с использованием нейронных сетей, где параметры модели настраиваются в процессе обучения. В записи $\pi(a|s,\mathbf{\theta})$ $\mathbf{\theta}$ это векторо параметров модели, например веса нейронной сети.


* __Цель обучения__ с подкреплением заключается в __нахождении оптимальной стратегии__, которая максимизирует ожидаемое вознаграждение (reward) в долгосрочной перспективе. 
* __Агент обучается, итеративно взаимодействуя с средой и обновляя свою стратегию__, чтобы лучше адаптироваться к окружающей среде и достигать своих целей.

__Определение вознаграждения__

* Траектория агента: ${(s_0, a_0), (s_1, a_1), \ldots}$ 

<center>         
    <img src="./img/mp1.png" alt="" style="width: 300px;"/>
    <b>It is a special case of a Markov Decision Process (MDP): A graph where each node is a particular game state and each edge is a possible (in general probabilistic) transition. Each edge also gives a reward, and the goal is to compute the optimal way of acting in any state to maximize rewards.</b> <br/>
</center>

* Суммарное дисконтированное вознаграждение тогда: 
$$R_t = \gamma^0 r(s_t, a_t) + \gamma^1 r(s_{t+1}, a_{t+1}) + \gamma^2 r(s_{t+2}, a_{t+2}) + \ldots$$
где $\gamma \in [0, 1]$ - дисконтирующий множитель

$$R_0 = \sum_{t=0}^{\infty} \gamma^t r_t$$

где:
- $R$ - дисконтированное вознаграждение.
- $t$ - шаг времени.
- $\gamma$ - коэффициент дисконтирования, обычно $\gamma \in [0, 1]$.
- $r_t$ - награда (reward) на шаге времени \(t\).

Это выражение представляет собой бесконечную сумму наград, взвешенных коэффициентами дисконтирования \(\gamma^t\), где \(\gamma\) определяет, как сильно будут учитываться будущие награды по сравнению с текущей.

__Функции V и Q__

__Функция значения состояния__ (value function, $V^\pi(s)$) - ожидаемая сумма дисконтированных будущих вознаграждений при следовании политике \(\pi\) если начать из состоянии $s$:

$$V^\pi(s) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t r_t \,|\, s_0 = s \right]$$

Где:
* $\mathbb{E}_\pi$ - оператор математического ожидания по политике $\pi$.
* $r_t$ - награда (reward) на временном шаге $t$.
* $s_0$ - начальное состояние.

__Q-функция__ - ожидаемая сумма дисконтированных будущих вознаграждений при следовании политике \(\pi\) если начать из состоянии $s$ при выполнении действия $a$:

$$Q^\pi(s, a) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t r_t \,|\, s_0 = s, a_0 = a \right]$$

* Если бы мы знали функции $V$ и $Q$ то можно было бы просто выбирать то действие $a$, которое максимизирует $Q(s,a)$.
* Т.е. функции $V$ и $Q$ — это как раз то, что нам нужно оценить.

__Уравнение Беллмана для функции значения состояния__

$$V^\pi(s) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t r_t \,|\, s_0 = s \right]=$$

первый член - это награда $r_0$, которая получается после выполнения действия $a_0$ в состоянии $s_0$:

$$= \mathbb{E}_\pi \left[ r_0  + \gamma \sum_{t=0}^{\infty} \gamma^t r_{t+1} \,|\, s_0 = s \right]$$

распишем матожидание при выполнении первого шага стратегии:

$$V^\pi(s) = \sum_a \pi(a|s) \sum_{s'} P(s' | s, a) \left[ R(s, a, s') + \gamma \mathbb{E}_\pi \left[\sum_{t=0}^{\infty} \gamma^t r_{t+1} \,|\, s_0 = s' \right] \right] $$

выражение во вторых квадратных скобках это функция значения состояния для нового состояния $s'$

$$= \sum_a \pi(a|s) \sum_{s'} P(s' | s, a) [R(s, a, s') + \gamma V^\pi(s')]$$

Т.е. получаем рекурентную зависимость которая является уравнением Беллмана:

$$V^\pi(s) = \sum_a \pi(a|s) \sum_{s'} P(s' | s, a) [R(s, a, s') + \gamma V^\pi(s')]$$

__Уравнение Беллмана__ играет важную роль в динамическом программировании: оно позволяет __рекурсивно вычислять оптимальные значения для всех состояний на основе оптимальных значений для более коротких траекторий__.
* Это ключевой элемент для поиска оптимальных стратегий в марковских процессах принятия решений и обучении с подкреплением. 

Уравнение Беллмана для функции значения состояния $V(s)$ выглядит следующим образом:

Пусть функция значения состояния при оптимальной стратегии: $V^{*}(s) = \max_{\pi} \mathbb{E} \left[ \sum_{t=0}^{\infty} \gamma^t r_t \right]$, тогда:

$$V^{*}(s) = \max_a \sum_{s'} P(s' | s, a) [R(s, a, s') + \gamma V^{*}(s')]$$

Где:
* $V(s)$ - значение состояния $s$, то есть ожидаемая сумма наград при следовании оптимальной стратегии из состояния $s$.
* $a$ - действие, выбранное в состоянии $s$ в соответствии с оптимальной стратегией.
* $s'$ - следующее состояние.
* $P(s' | s, a)$ - вероятность перехода из состояния $s$ в состояние $s'$ при выполнении действия $a$.
* $R(s, a, s')$ - вознаграждение (reward), получаемая после выполнения действия $a$ в состоянии $s$ и перехода в состояние $s'$.
* $\gamma$ - коэффициент дисконтирования.

__Два подхода к нахождению оптимальной стратегии__

Tеоретически для того чтобы найти значения $V(s)$, можно просто решить систему линейных уравнений, неизвестными в которых являются $V(s)$ для разных состояний $s$.

Но __для этого нужно знать все параметры марковского процесса__, то есть функции R и P, а с этим в реальных ситуациях все сложно. 
* Все, что у нас обычно есть на входе — __окружающая среда, выдающая вознаграждения как черный ящик__. Т.е. в реальных задачах функции $R$ и $P$ тоже приходится обучать.

Методы обучения с подкреплением делятся на те, которые:
* обучают функции $R$ и $P$ в явном виде
* обходятся без этого и сразу обучают $V$ и/или $Q$.

__Классификация алгоритмов RL__

<center>         
    <img src="./img/rl_alg.png" alt="" style="width: 700px;"/>
    <b>Классификация алгоритмов RL</b> <br/>
</center>

<center>         
    <img src="./img/rl2.png" alt="" style="width: 400px;"/>
    <b>Подходы к RL</b> <br/>
</center>

# Метод Policy Gradient

__Введение в метод Policy Gradient__

__Policy gradient__ Класс методов reinforcement learning в которых стратегию $\pi_{\theta}(a|s)$ оптимизируют напрямую (в отличие от __Q-learning__ ). 

Интуитивное объяснение принципа работы:

$p_{\theta}(\tau)$ - вероятность того, что будет реализован сценарий $\tau$ при условии параметров модели $\theta$, т. е. функция правдоподобия. Нам хочется:
* увеличить правдоподобие "хороших" сценариев (обладающих высоким $R_{\tau}$)
* понизить правдоподобие "плохих" сценариев (с низким $R_{\tau}$).

__Игра Pong__

Игра Pong: https://www.youtube.com/watch?v=fiShX2pTz9A 

<center>         
    <img src="./img/pong1.png" alt="" style="width: 500px;"/>
    <b>Нейронная сеть для обучения игре Pong</b> <br/>
</center>

* На нейроны подается информация __о разности__ значений пикселей в последующих кадрах игры.
* Выходной нейрон - вероятность двигать ракетку вверх (иначе двигаем ракетку вниз).

__Принцип обучения методом Policy Gradient__

<center>         
    <img src="./img/pg1.png" alt="" style="width: 800px;"/>
    <b>Процесс обучения при обучении с учителем</b> <br/>
</center>

При обучении с учителем на каждом шаге мы знаем правильный ответ (правильный класс: "вверх" или "вниз"), на основании которого и проводим обратное распространение ошибки.

<center>         
    <img src="./img/pg2.png" alt="" style="width: 800px;"/>
    <b>Процесс обучения в Policy Gradient</b> <br/>
</center>

* У нас нет правильного ответа для каждого шага игры, есть только ответ для всей игры: "выиграл" или "проиграл".
* __В случае выигрыша__ действие на каждом шаге (выбранный класс) поощряется: устанавливается что оно было верным.
* __В случае проигрыша__ действие на каждом шаге (выбранный класс) штрафуется: устанавливается что оно было НЕ верным.

__Пример сценариев обучения в Policy Gradient__

* __Каждая точка__ представляет собой некоторое игровое __состояние__ (три примера состояния визуализированы внизу)
* __Каждая стрелка__ - это переход (шаг по времени): действие (указано у стрелки) и изменение состояния.

<center>         
    <img src="./img/pg3.png" alt="" style="width: 700px;"/>
    <b>Пример 4 тректорий игры в Pong</b> <br/>
</center>

* В этих примерах мы выиграли 2 игры и проиграли 2 игры. 

Используя Policy Gradients:
* мы поощряем две выигранные игры: слегка поощряем каждое действие, которое мы совершили в этом эпизоде. 
* мы штрафуем  две проигранные игры: слегка штрафуем каждое действие, которое мы совершили в этом эпизоде.


* Pong AI with Policy Gradients https://www.youtube.com/watch?v=YOW8m2YGtRg&list=TLGG2Hi1VyMnZ7gwMjExMjAyMw
    * The learned agent (in green, right) facing off with the hard-coded AI opponent (left).

__Loss Policy Gradients__

Обозначения:
* $p(x)=p(x|s,\theta)$ - policy function, $\theta$ - веса сети
* $f(x)$ - вознаграждения 

По
<center>         
    <img src="./img/pg4.png" alt="" style="width: 700px;"/>
    <b>...</b> <br/>
</center>

* Что весьма похоже на Cross-entropy loss

__Алгоритм обучения с помощью Policy Gradients__
1. Разыгрываем N игр при помощи имеющейся нейросети реализующей стратегию $\pi(a|s)$ - получаем сценарии игр для обучения.
2. На основе Loss Policy Gradients по полученным сценариям игр обучаем нейросеть стратегии.
3. Меняем нейросеть стратегии $\pi(a|s)$  на обученную, переходим к п.1 


Ссылка: http://karpathy.github.io/2016/05/31/rl/

<center>         
    <img src="./img/pg5.png" alt="" style="width: 700px;"/>
    <b>...</b> <br/>
</center>
<br/>

<center>         
    <img src="./img/pg6.png" alt="" style="width: 700px;"/>
    <b>...</b> <br/>
</center>
<br/>

<center>         
    <img src="./img/pg7.png" alt="" style="width: 700px;"/>
    <b>Baseline</b> <br/>
</center>

__Преимущества Policy gradient:__

* __Легко обобщается на задачи с большим множеством действий___, в том числе на задачи с непрерывным множеством действий;
* По большей части __избегает конфликта между эксплуатацией (exploitation) и исследованием (exploration)__, так как оптимизирует напрямую стохастическую стратегию $\pi_{\theta}(a|s)$;
* Имеет __более сильные гарантии сходимости__: если Q-learning гарантированно сходится только для МППР с конечными множествами действий и состояний, то policy gradient сходится к локальному оптимуму всегда, в том числе в случае бесконечных множеств действий и состояний.

__Недостатки Policy gradient:__

* __Очень низкая скорость работы__ - требуется большое количество вычислений для оценки $\nabla_{\theta} J(\theta)$ по методу Монте-Карло, так как:
     * для получения всего одного семпла требуется произвести $T$ взаимодействий со средой;
     * случайная величина $\nabla_{\theta} \log p_{\theta}(\tau) R_{\tau}$ имеет большую дисперсию поэтому для точной оценки $\nabla_{\theta} J(\theta) = E_{\tau \sim p_{\theta}(\tau)} \left[ \nabla_{\theta} \log p_{\theta}(\tau) R_{\tau} \right]$ требуется много семплов;
    * __cемплы__, собранные для предыдущих значений $\theta$, __никак не переиспользуются на следующем шаге__, семплирование нужно делать заново на каждом шаге градиентного спуска.
* В случае конечных марковских процессов принятия решений (МППР) Q-learning сходится к глобальному оптимуму, тогда как __policy gradient может застрять в локальном__.

## Actor-Critic

В модели Actor-Critic обучается функции значения состояния $b=V(s)$ предсказывающая дисконтированное вознаграждение по текущему состоянию:

<center>         
    <img src="./img/ac2.png" alt="" style="width: 700px;"/>
    <b>Baseline</b> <br/>
</center>

Обучение $b=V(s)$ проводится параллельно собучением Policy gradient рассматривающей не исходое вознаграждение а дельту вознаграждения ( $\log p(a) \sum_i \left( \gamma^i r_{s+i} - V\right)$ ) и ожидаемого значения состояния для текущего состояния:

<center>         
    <img src="./img/ac1.png" alt="" style="width: 700px;"/>
    <b>Baseline</b> <br/>
</center>

## Q-Learning

__Уравнение Беллмана для функции значения состояния__

Уравнение Беллмана для функции значения состояния $V(s)$ выглядит следующим образом:

Пусть функция значения состояния при оптимальной стратегии: $V^{*}(s) = \max_{\pi} \mathbb{E} \left[ \sum_{t=0}^{\infty} \gamma^t r_t \right]$, тогда:

$$V^{*}(s) = \max_a \sum_{s'} P(s' | s, a) [R(s, a, s') + \gamma V^{*}(s')]$$

Где:
* $V(s)$ - значение состояния $s$, то есть ожидаемая сумма наград при следовании оптимальной стратегии из состояния $s$.
* $a$ - действие, выбранное в состоянии $s$ в соответствии с оптимальной стратегией.
* $s'$ - следующее состояние.
* $P(s' | s, a)$ - вероятность перехода из состояния $s$ в состояние $s'$ при выполнении действия $a$.
* $R(s, a, s')$ - вознаграждение (reward), получаемая после выполнения действия $a$ в состоянии $s$ и перехода в состояние $s'$.
* $\gamma$ - коэффициент дисконтирования.

__Уравенение Беллмана для функции Q__

__Q-функция__ - ожидаемая сумма дисконтированных будущих вознаграждений при следовании политике \(\pi\) если начать из состоянии $s$ при выполнении действия $a$:

$$Q^\pi(s, a) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t r_t \,|\, s_0 = s, a_0 = a \right]$$

* Если бы мы знали функции $V$ и $Q$ то можно было бы просто выбирать то действие $a$, которое максимизирует $Q(s,a)$.
* Т.е. функции $V$ и $Q$ — это как раз то, что нам нужно оценить.

<center>         
    <img src="./img/ql1.png" alt="" style="width: 500px;"/>
    <b>___</b> <br/>
</center>

__Алгоритм DQL__

<center>         
    <img src="./img/ql3.png" alt="" style="width: 300px;"/>
    <b>Схема нейросети Q для игр на Atari</b> <br/>
</center>


<center>         
    <img src="./img/ql2.png" alt="" style="width: 700px;"/>
    <b>___</b> <br/>
</center>

<center>         
    <img src="./img/ql4.png" alt="" style="width: 400px;"/>
    <b>Работа с памятью эпизодов</b> <br/>
</center>

__Сравнение подходов__

<center>         
    <img src="./img/comp1.png" alt="" style="width: 600px;"/>
    <b>Сравнение подходов</b> <br/>
</center>

---
# Спасибо за внимание!

---
### Технический раздел:

* https://neerc.ifmo.ru/wiki/index.php?title=Методы_policy_gradient_и_алгоритм_асинхронного_актора-критика
* https://neerc.ifmo.ru/wiki/index.php?title=Обучение_с_подкреплением

методы, которые позволяют оптимизировать стратегию $\pi_{\theta}(s|a)$ напрямую. Такие алгоритмы относятся к классу алгоритмов ''policy gradient''.

Интуитивное объяснение принципа работы:

$p_{\theta}(\tau)$ - вероятность того, что будет реализован сценарий $\tau$ при условии параметров модели $\theta$, т. е. функция правдоподобия. Нам хочется увеличить правдоподобие "хороших" сценариев (обладающих высоким $R_{\tau}$) и понизить правдоподобие "плохих" сценариев (с низким $R_{\tau}$).


__Policy gradient__ Класс методов reinforcement learning в которых стратегию $\pi_{\theta}(s|a)$ оптимизируют напрямую (в отличие от _Q-learning_ ). 

Интуитивное объяснение принципа работы:

$p_{\theta}(\tau)$ - вероятность того, что будет реализован сценарий $\tau$ при условии параметров модели $\theta$, т. е. функция правдоподобия. Нам хочется увеличить правдоподобие "хороших" сценариев (обладающих высоким $R_{\tau}$) и понизить правдоподобие "плохих" сценариев (с низким $R_{\tau}$).

Двигаясь вверх по градиенту функции полного выигрыша мы повышаем логарифм функции правдоподобия для сценариев, имеющих больший положительный $R_{\tau}$.

Преимущества Policy gradient:

* Легко обобщается на задачи с большим множеством действий, в том числе на задачи с непрерывным множеством действий;
* По большей части избегает конфликта между эксплуатацией (exploitation) и исследованием (exploration), так как оптимизирует напрямую стохастическую стратегию $\pi_{\theta}(a|s)$;
* Имеет более сильные гарантии сходимости: если Q-learning гарантированно сходится только для МППР с конечными множествами действий и состояний, то policy gradient сходится к локальному оптимуму всегда, в том числе в случае бесконечных множеств действий и состояний.

Недостатки Policy gradient:

* Очень низкая скорость работы - требуется большое количество вычислений для оценки $\nabla_{\theta} J(\theta)$ по методу Монте-Карло, так как:
     * для получения всего одного семпла требуется произвести $T$ взаимодействий со средой;
     * случайная величина $\nabla_{\theta} \log p_{\theta}(\tau) R_{\tau}$ имеет большую дисперсию поэтому для точной оценки $\nabla_{\theta} J(\theta) = E_{\tau \sim p_{\theta}(\tau)} \left[ \nabla_{\theta} \log p_{\theta}(\tau) R_{\tau} \right]$ требуется много семплов;
    * cемплы, собранные для предыдущих значений $\theta$, никак не переиспользуются на следующем шаге, семплирование нужно делать заново на каждом шаге градиентного спуска.
* В случае конечных марковских процессов принятия решений (МППР) Q-learning сходится к глобальному оптимуму, тогда как policy gradient может застрять в локальном.

<br/> next <em class="qs"></em> qs line 
<br/> next <em class="an"></em> an line 
<br/> next <em class="nt"></em> an line 
<br/> next <em class="df"></em> df line 
<br/> next <em class="ex"></em> ex line 
<br/> next <em class="pl"></em> pl line 
<br/> next <em class="mn"></em> mn line 
<br/> next <em class="plmn"></em> plmn line 
<br/> next <em class="hn"></em> hn line 